# Finetuing

## module import


In [1]:
import wandb
import time
import numpy as np
import pandas as pd

import tensorflow as tf
import torch

import re
import os

import datasets
from datasets import load_dataset, load_metric, ClassLabel, Sequence, Dataset

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split as tts

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, AutoModelForMaskedLM
import matplotlib.pyplot as plt

print('torch version:', torch.__version__)
print('tf version:', tf.__version__)

2023-04-09 07:35:21.029123: I tensorflow/core/tpu/tpu_api_dlsym_initializer.cc:116] Libtpu path is: libtpu.so
[percpu.cc : 539] RAW: rseq syscall failed with errno 22 after membarrier sycall succeeded.


torch version: 1.10.0+cu102
tf version: 2.8.0


In [2]:
import torch

In [3]:
# using TPU through torch
import torch_xla
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.utils.serialization as xser
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

print(torch_xla.__version__)

1.10


In [4]:
# random seed fix
import random

random.seed(2022)
torch.manual_seed(2022)
np.random.seed(2022)

### TPU setting

In [5]:
## Google cloud project에서 TPU 셋팅

# .py로 실행할 때 TPU 셋팅 명령어
#!export XRT_TPU_CONFIG="localservice;0;localhost:51011"

# 주피터 노트 또는 주피터 랩에서 실행할 때, TPU 셋팅 명령어
import os
os.environ['XRT_TPU_CONFIG'] = "localservice;0;localhost:51011"
device = xm.xla_device()
device

2023-04-09 07:35:24.475751: E tensorflow/core/framework/op_kernel.cc:1676] OpKernel ('op: "TPURoundRobin" device_type: "CPU"') for unknown op: TPURoundRobin
2023-04-09 07:35:24.475804: E tensorflow/core/framework/op_kernel.cc:1676] OpKernel ('op: "TpuHandleToProtoKey" device_type: "CPU"') for unknown op: TpuHandleToProtoKey


device(type='xla', index=1)

## data load & EDA

In [6]:
cd ~/sonny

/home/yejin/sonny


In [7]:
pwd

'/home/yejin/sonny'

In [8]:
ls

'015.감성 및 발화 스타일별 음성합성 데이터'@
 KEMDy19@
 KEMDy19.7z@
 KEMDy20@
 KEMDy20.7z@
 KEMDy20_test_data.csv@
 KEMDy20_train_data.csv@
 KEMDy20_val_data.csv@
 RoBERTa_batch128dr/
 RoBERTa_batch8/
 RoBERTa_batch8dr/
 balance_train.csv@
 dataset@
 dataset-20230325T131448Z-001.zip@
 dataset-20230325T131448Z-002.zip@
 emotion_aihub@
 lost+found@
 nlp_emotion/
 utils/
 wandb/
 wav_data@
 wav_data.7z@


## input data transform

In [9]:
# load DAPT model, tokenizer
num_labels = 7
model = AutoModelForSequenceClassification.from_pretrained('klue/roberta-large', num_labels=num_labels)
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classif

In [10]:
from utils.Customdataset import CustomDataset
from torch.utils.data import Dataset, DataLoader
from torch import nn

train_dataset = CustomDataset('balance_train.csv')
val_dataset = CustomDataset('KEMDy20_val_data.csv')
test_dataset = CustomDataset('KEMDy20_test_data.csv')


/home/yejin/sonny/utils/Customdataset.py:43: DtypeWarning: Columns (3,7,10,13,16,19,22,25,28,31,34) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(csv_path, header=[0, 1])


In [11]:
train_dataset[1],train_dataset[2],train_dataset[0],

(('신세 진 것은 꼭 갚아야겠다고 했다.', 0),
 ('뒤이어 흐느껴 우는 소리가 들렸습니다.', 6),
 ('그에게 불쌍한 그녀가 계속 보였다.', 6))

In [12]:
print(len(train_dataset))
print(len(val_dataset))

70018
2000


In [13]:
# train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers = 3, collate_fn=preprocess_function)
# val_loader = DataLoader(val_dataset, batch_size=2, shuffle=True, num_workers = 3, collate_fn=preprocess_function)

# class_weights = [1/1274,1/180,1/197,1/9098,1/94,1/53,1/182]
# criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weights))

## eval metric 정의

In [14]:
'''
multi sentiment analysis는 기본적으로 문장 분류 문제
KLUE task를 학습한 klue-roberta 중에서 TC(topic classification)와 유사(완전히 같지는 않음)
TC는 평가 지표로 macro f1 score를 사용

but klue에서는 다중감성분류를 학습하지 않았고,
glue에서는 sst2라는 긍부정 분류 task가 있는데 accuracy를 사용

but 우리는 데이터셋이 다소 imbalance하기 때문에 accuracy를 채택하면 오차가 발생할 수 있음

따라서 더 data imbalance에 robust한 metric를 만들기 위해
sklearn에서 제공하는 f1 score metric을 활용
'''

def eval_metric(pred, real):
    f1 = {
        'macro_f1':f1_score(real, pred, average='macro'),
        'micro_f1':f1_score(real, pred, average='micro'),
        'weighted_f1':f1_score(real, pred, average='weighted'), 
    }
    return f1

In [15]:
# metric test
np.random.seed(2022)
fake_preds = np.random.randint(0, 6, size=(64,))
fake_labels = np.random.randint(0, 6, size=(64,))
fake_preds, fake_labels

(array([5, 4, 5, 0, 1, 1, 0, 0, 2, 0, 0, 5, 1, 1, 3, 3, 3, 0, 3, 0, 3, 0,
        5, 0, 2, 2, 3, 1, 2, 0, 5, 5, 1, 5, 5, 4, 0, 2, 2, 2, 3, 4, 5, 1,
        2, 4, 0, 5, 5, 4, 2, 5, 0, 5, 5, 5, 5, 5, 0, 3, 3, 2, 4, 0]),
 array([1, 4, 1, 0, 5, 5, 0, 0, 3, 1, 5, 3, 2, 1, 5, 2, 1, 4, 1, 2, 2, 1,
        5, 1, 2, 1, 4, 3, 0, 4, 4, 4, 4, 0, 1, 2, 2, 4, 1, 5, 5, 2, 0, 0,
        4, 0, 0, 3, 4, 3, 1, 2, 3, 2, 3, 2, 3, 4, 5, 3, 3, 5, 3, 4]))

In [16]:
eval_metric(fake_preds, fake_labels)

{'macro_f1': 0.15364479575005888,
 'micro_f1': 0.15625,
 'weighted_f1': 0.1490065307499518}

In [17]:
# eval metric
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    result = eval_metric(predictions, labels)
    return result

### Model to tpu

In [18]:
model = model.to(device)

## train arg

In [19]:
# preprocessing function
def preprocess_function(data):
    data_T = zip(*data).__iter__()
    out = {**tokenizer(
        # tokenizing
        next(data_T),
        max_length=512,

        # 최대 길이보다 긴 시퀀스는 최대 길이에 맞춰 자름
        truncation=True,

        # tokenizer가 token_type_ids를 return하지 않게 함
        # roberta는 필요없기 때문
        return_token_type_ids=False,
        return_tensors="pt",
        padding=True,
    ), "labels":torch.tensor(next(data_T))
    }
    # for i in list(out):
    #     out[i]=out[i].to(device)
    return out

In [20]:
# wandb.finish()

In [21]:
%env WANDB_NOTEBOOK_NAME = /home/yejin/sonny/nlp_emotion/Finetuning.ipynb

env: WANDB_NOTEBOOK_NAME=/home/yejin/sonny/nlp_emotion/Finetuning.ipynb


In [ ]:
start = time.time()

# metric_name은 eval_metric에서 리턴받는 dict 형태의 키 이름
metric_name = "micro_f1"

# batch size 지정
batch_size = 32
num_train_epochs = 15




# Project name and run name
run_name = f"RoBERTa_emotion_batch{batch_size}"
%env WANDB_PROJECT = our_emotion

# path 설정
trained_model_path = f"ro{batch_size}"


args = TrainingArguments(
    output_dir=trained_model_path,
    overwrite_output_dir=True,
    # evaluation_strategy="epoch",
    evaluation_strategy="steps",
    learning_rate=1e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    save_total_limit=2,
    tpu_num_cores = 85,
    seed = 2023,
    data_seed = 2023,
    dataloader_pin_memory = True,
    report_to="wandb",
    run_name=run_name,
    optim = 'adafactor',
    logging_steps=10,
    eval_steps=200,
    save_steps=200
)
 
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=15)],
    data_collator=preprocess_function
)

trainer.train()

print("time :", time.time() - start)
wandb.finish()

env: WANDB_PROJECT=our_emotion


Step,Training Loss,Validation Loss,Macro F1
200,1.967900,2.022730,0.007063
400,1.932800,1.724668,0.119862


In [25]:
trainer.save_model(trained_model_path)

### Upload hugging face

In [26]:
# load model, tokenizer
# batch_size = 32 
# model = AutoModelForSequenceClassification.from_pretrained('jungyong/FT_batch32_lyric')
# tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')

MODEL_SAVE_REPO = f'FT_lcs_{run_name}'
HUGGINGFACE_AUTO_TOKEN = input() # https://huggingface.co/settings/token
 
## Push to huggingface-hub
model.push_to_hub(
    MODEL_SAVE_REPO, 
    use_temp_dir=True, 
    use_auth_token=HUGGINGFACE_AUTO_TOKEN
)

/home/vazz0901/.local/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:38: FutureWarning: Deprecated positional argument(s) used in 'create_repo': pass token='FT_lcs_adafactor_lr1e_6' as keyword args. From version 0.12 passing these as positional arguments will result in an error,
  warnings.warn(
/home/vazz0901/.local/lib/python3.8/site-packages/huggingface_hub/hf_api.py:681: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning https://huggingface.co/JUNEYEOB/FT_lcs_adafactor_lr1e_6 into local empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Configuration saved in /tmp/tmpjnokmc9g/config.json
tcmalloc: large alloc 1469087744 bytes == 0x1d10084000 @  0x7f59cc76c680 0x7f59cc78cda2 0x5f8dfc 0x64f870 0x527012 0x5c64c0 0x5f4cc1 0x5f4f85 0x486664 0x539ccb 0x539bf9 0x66321b 0x53a821 0x53a01f 0x6632cc 0x53a164 0x53a01f 0x66321b 0x53a164 0x53a8d8 0x66134d 0x6615f0 0x505166 0x56bbfa 0x569dba 0x5f6eb3 0x56bacd 0x569dba 0x5f6eb3 0x56bacd 0x569dba
tcmalloc: large alloc 1836359680 bytes == 0x1c6c32c000 @  0x7f59cc76c680 0x7f59cc78cda2 0x5f8dfc 0x64f870 0x527012 0x5c64c0 0x5f4cc1 0x5f4f85 0x486664 0x539ccb 0x539bf9 0x66321b 0x53a821 0x53a01f 0x6632cc 0x53a164 0x53a01f 0x66321b 0x53a164 0x53a8d8 0x66134d 0x6615f0 0x505166 0x56bbfa 0x569dba 0x5f6eb3 0x56bacd 0x569dba 0x5f6eb3 0x56bacd 0x569dba
tcmalloc: large alloc 2295455744 bytes == 0x1d52c28000 @  0x7f59cc76c680 0x7f59cc78cda2 0x5f8dfc 0x64f870 0x527012 0x5c64c0 0x5f4cc1 0x5f4f85 0x486664 0x539ccb 0x539bf9 0x66321b 0x53a821 0x53a01f 0x6632cc 0x53a164 0x53a01f 0x66321b 0x53a164 0x53a8d8 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Upload file pytorch_model.bin:   0%|          | 32.0k/1.88G [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/JUNEYEOB/FT_lcs_adafactor_lr1e_6
   86e9f45..fb805fc  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'https://huggingface.co/JUNEYEOB/FT_lcs_adafactor_lr1e_6/commit/fb805fcd88543d8e9b455a137be7fbd0867e249f'

In [5]:
!git clone https://huggingface.co/JUNEYEOB/FT_lcs_adafactor_lr1e_6

Cloning into 'FT_lcs_adafactor_lr1e_6'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), 1.29 KiB | 1.29 MiB/s, done.
tcmalloc: large alloc 1471086592 bytes == 0x560e8acde000 @  0x7f12d2724680 0x7f12d2744bdd 0x560e1c0f06b5 0x560e1c0bd840 0x560e1c05b6a1 0x560e1bfe496a 0x560e1bfe51a2 0x560e1bfe5bf2 0x560e1c009f3c 0x560e1c00a4e4 0x560e1c00ab32 0x560e1c0e4eac 0x560e1bf13f2c 0x560e1bef42f4 0x560e1bef53b4 0x560e1bef3e9e 0x7f12d2468083 0x560e1bef3f0e
tcmalloc: large alloc 2206621696 bytes == 0x560ee27ce000 @  0x7f12d2724680 0x7f12d2744bdd 0x560e1c0f06b5 0x560e1c0bd840 0x560e1c05b6a1 0x560e1bfe496a 0x560e1bfe51a2 0x560e1bfe5bf2 0x560e1c009f3c 0x560e1c00a4e4 0x560e1c00ab32 0x560e1c0e4eac 0x560e1bf13f2c 0x560e1bef42f4 0x560e1bef53b4 0x560e1bef3e9e 0x7f12d2468083 0x560e1bef3f0e


In [ ]:
!cd FT_lcs_adafactor_lr1e_6
!cp -r ~/batch